# Daten Extraktion und Transformation

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import sqlite3
import matplotlib.pyplot as plt
import geopandas as gpd
import datetime, time

## Gesamtüberblick Marktstammdatenregister

Im Marktstammdatenregister werden alle Stromerzeuger (Kraftwerke) Deutschlands aufgeführt, um Stromerzeugungskapazitäten aufzuzeigen.   
Download-Link: https://www.marktstammdatenregister.de/MaStR/Datendownload     
Stand: 1.07.2023

In [2]:
# Ansicht Katalogkategorien
katalog_kat = pd.read_xml("../input/Marktstammdaten/Katalogkategorien.xml", encoding='utf-16', parser='lxml')
katalog_kat

,Id,Name
0,1,Energieträger
1,2,Sparte
2,3,Anlagenart
3,4,Betriebsstatus
4,5,Nachweisart
...,...,...
110,175,NBPStatusFilter
111,187,Verbrennung Wasserstoff
112,189,Webhilfe Element-Status
113,190,Webhilfe Dokumenten-Status


In [3]:
# Ansicht Katalogwerte
katalog_wert = pd.read_xml("../input/Marktstammdaten/Katalogwerte.xml", encoding='utf-16', parser='lxml')
katalog_wert

,Id,Wert,KatalogKategorieId
0,1,andere Gase,1
1,2,Biomasse,1
2,3,Braunkohle,1
3,5,Erdgas,1
4,6,Geothermie,1
...,...,...,...
1641,1001679,Siemens Gamesa Renewable Energy GmbH & Co. KG,74
1642,1001680,Amtsgericht Wesel,93
1643,1001681,MAX-wyn GmbH,74
1644,1001682,Nordex Germany GmbH,74


## Photovoltaik Anlagen laut Marktstammdatenregister

### Dateien einlesen

In [4]:
# alle EinheitenSolar Dateien als Gesamt-Tabelle einlesen.
filenames = glob.glob("../input/Marktstammdaten/EinheitenSolar*.xml")
df_pv = pd.concat(map(lambda file: pd.read_xml(file, encoding="utf-16", parser="lxml",
                                               dtype={"Gemeindeschluessel": str, "Postleitzahl": str}), filenames))
df_pv

,EinheitMastrNummer,DatumLetzteAktualisierung,LokationMaStRNummer,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Land,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,...,InAnspruchGenommeneAckerflaeche,GeplantesInbetriebnahmedatum,GenMastrNummer,Einsatzverantwortlicher,WeicDisplayName,DatumBeginnVoruebergehendeStilllegung,DatumWiederaufnahmeBetrieb,Buergerenergie,Kraftwerksnummer,Weic
0,SEE984033548619,2020-02-20T16:28:35.2508122,SEL948991715391,1,ABR949444220202,84,1409.0,Münster,Münster,05515000,...,NaN,None,None,NaN,None,None,None,NaN,None,NaN
1,SEE901901460125,2021-07-15T07:54:37.8637095,SEL982068309366,1,ABR930055871044,84,1402.0,Ostalbkreis,Schwäbisch Gmünd,08136065,...,NaN,None,None,NaN,None,None,None,NaN,None,NaN
2,SEE983679054270,2019-05-28T08:26:38.0316692,SEL906699064968,1,ABR983885950021,84,1400.0,Havelland,Nauen,12063208,...,NaN,None,None,NaN,None,None,None,NaN,None,NaN
3,SEE978732598938,2020-12-07T08:19:07.9860793,SEL996128012264,1,ABR980052996196,84,1403.0,Regensburg,Pentling,09375180,...,NaN,None,None,NaN,None,None,None,NaN,None,NaN
4,SEE970592691989,2021-01-25T11:18:30.0433603,SEL975715515692,1,ABR970373690446,84,1412.0,Saarlouis,Saarlouis,10044115,...,NaN,None,None,NaN,None,None,None,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,SEE998635136235,2020-07-14T07:21:05.4509560,SEL932897235060,1,ABR941126007873,84,1409.0,Siegen-Wittgenstein,Bad Laasphe,05970028,...,NaN,None,None,NaN,NaN,None,None,NaN,None,NaN
99996,SEE978483083905,2020-04-29T09:37:03.4024856,SEL942528518813,0,ABR903375397990,84,1409.0,Hochsauerlandkreis,Bestwig,05958008,...,NaN,None,None,NaN,NaN,None,None,NaN,None,NaN
99997,SEE938515036866,2020-11-26T08:57:11.0060494,SEL938123450887,1,ABR985070432688,84,1403.0,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,...,NaN,None,None,NaN,NaN,None,None,NaN,None,NaN
99998,SEE910101482608,2020-05-02T10:16:32.7818459,SEL972227303496,1,ABR925833676461,84,1405.0,Groß-Gerau,Riedstadt,06433011,...,NaN,None,None,NaN,NaN,None,None,NaN,None,NaN


In [5]:
pv_raw = df_pv.copy()
pv_raw.reset_index(drop=True,inplace=True)
pv_raw

,EinheitMastrNummer,DatumLetzteAktualisierung,LokationMaStRNummer,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Land,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,...,InAnspruchGenommeneAckerflaeche,GeplantesInbetriebnahmedatum,GenMastrNummer,Einsatzverantwortlicher,WeicDisplayName,DatumBeginnVoruebergehendeStilllegung,DatumWiederaufnahmeBetrieb,Buergerenergie,Kraftwerksnummer,Weic
0,SEE984033548619,2020-02-20T16:28:35.2508122,SEL948991715391,1,ABR949444220202,84,1409.0,Münster,Münster,05515000,...,NaN,None,None,None,None,None,None,NaN,None,None
1,SEE901901460125,2021-07-15T07:54:37.8637095,SEL982068309366,1,ABR930055871044,84,1402.0,Ostalbkreis,Schwäbisch Gmünd,08136065,...,NaN,None,None,None,None,None,None,NaN,None,None
2,SEE983679054270,2019-05-28T08:26:38.0316692,SEL906699064968,1,ABR983885950021,84,1400.0,Havelland,Nauen,12063208,...,NaN,None,None,None,None,None,None,NaN,None,None
3,SEE978732598938,2020-12-07T08:19:07.9860793,SEL996128012264,1,ABR980052996196,84,1403.0,Regensburg,Pentling,09375180,...,NaN,None,None,None,None,None,None,NaN,None,None
4,SEE970592691989,2021-01-25T11:18:30.0433603,SEL975715515692,1,ABR970373690446,84,1412.0,Saarlouis,Saarlouis,10044115,...,NaN,None,None,None,None,None,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3180108,SEE998635136235,2020-07-14T07:21:05.4509560,SEL932897235060,1,ABR941126007873,84,1409.0,Siegen-Wittgenstein,Bad Laasphe,05970028,...,NaN,None,None,None,None,None,None,NaN,None,None
3180109,SEE978483083905,2020-04-29T09:37:03.4024856,SEL942528518813,0,ABR903375397990,84,1409.0,Hochsauerlandkreis,Bestwig,05958008,...,NaN,None,None,None,None,None,None,NaN,None,None
3180110,SEE938515036866,2020-11-26T08:57:11.0060494,SEL938123450887,1,ABR985070432688,84,1403.0,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,...,NaN,None,None,None,None,None,None,NaN,None,None
3180111,SEE910101482608,2020-05-02T10:16:32.7818459,SEL972227303496,1,ABR925833676461,84,1405.0,Groß-Gerau,Riedstadt,06433011,...,NaN,None,None,None,None,None,None,NaN,None,None


**Die Datei pv_raw enthält 3.180.113 Zeilen und 66 Spalten.**

In [6]:
print(pv_raw.dtypes)

EinheitMastrNummer                        object
DatumLetzteAktualisierung                 object
LokationMaStRNummer                       object
NetzbetreiberpruefungStatus                int64
AnlagenbetreiberMastrNummer               object
                                          ...   
DatumBeginnVoruebergehendeStilllegung     object
DatumWiederaufnahmeBetrieb                object
Buergerenergie                           float64
Kraftwerksnummer                          object
Weic                                      object
Length: 66, dtype: object


In [7]:
# Alle Kategorien einblenden
pd.set_option('display.max_columns', 66)
pv_raw

,EinheitMastrNummer,DatumLetzteAktualisierung,LokationMaStRNummer,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Land,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitSystemstatus,EinheitBetriebsstatus,NichtVorhandenInMigriertenEinheiten,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Energietraeger,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,EegMaStRNummer,Nebenausrichtung,NebenausrichtungNeigungswinkel,Strasse,StrasseNichtGefunden,Hausnummer_nv,HausnummerNichtGefunden,Laengengrad,Breitengrad,FernsteuerbarkeitDr,Hausnummer,Adresszusatz,FernsteuerbarkeitDv,NetzbetreiberpruefungDatum,DatumDesBetreiberwechsels,DatumRegistrierungDesBetreiberwechsels,Gemarkung,FlurFlurstuecknummern,AnschlussAnHoechstOderHochSpannung,InAnspruchGenommeneFlaeche,ArtDerFlaecheIds,DatumEndgueltigeStilllegung,InAnspruchGenommeneAckerflaeche,GeplantesInbetriebnahmedatum,GenMastrNummer,Einsatzverantwortlicher,WeicDisplayName,DatumBeginnVoruebergehendeStilllegung,DatumWiederaufnahmeBetrieb,Buergerenergie,Kraftwerksnummer,Weic
0,SEE984033548619,2020-02-20T16:28:35.2508122,SEL948991715391,1,ABR949444220202,84,1409.0,Münster,Münster,05515000,48147,Münster,2019-02-01,2007-07-20,472,35,0,Photovoltaikanlage ERWin4,0,0,2495,3.960,3.960,0.0,688.0,4.00,1450.0,22.0,853.0,802.0,1.0,699.0,809.0,713.0,EEG920083771065,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,None,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,None,None,NaN,None,NaN
1,SEE901901460125,2021-07-15T07:54:37.8637095,SEL982068309366,1,ABR930055871044,84,1402.0,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,2013-01-31,472,35,0,man-power,0,0,2495,7.410,7.410,0.0,689.0,8.30,1450.0,38.0,853.0,803.0,1.0,699.0,809.0,713.0,EEG943811805092,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,None,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,None,None,NaN,None,NaN
2,SEE983679054270,2019-05-28T08:26:38.0316692,SEL906699064968,1,ABR983885950021,84,1400.0,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,2016-02-19,472,35,0,eigene PVA,0,0,2495,5.040,5.000,0.0,689.0,5.00,1449.0,16.0,853.0,803.0,1.0,699.0,809.0,713.0,EEG963355832040,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,None,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,None,None,NaN,None,NaN
3,SEE978732598938,2020-12-07T08:19:07.9860793,SEL996128012264,1,ABR980052996196,84,1403.0,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,2016-12-16,472,35,0,Photovoltaik EFH Löbert - Mattinger Weg 6,0,0,2495,6.360,6.000,0.0,689.0,6.00,1448.0,24.0,853.0,1535.0,1.0,700.0,809.0,713.0,EEG957216544098,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,None,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,None,None,NaN,None,NaN
4,SEE970592691989,2021-01-25T11:18:30.0433603,SEL975715515692,1,ABR970373690446,84,1412.0,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,2011-12-08,472,35,0,"PV-Anlage 7,2 (Voll)",0,0,2495,7.200,7.200,0.0,689.0,7.60,1450.0,30.0,853.0,802.0,0.0,701.0,809.0,713.0,EEG932405876693,697.0,809.0,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,None,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,None,None,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3180108,SEE998635136235,2020-07-14T07:21:05.4509560,SEL932897235060,1,ABR941126007873,84,1409.0,Siegen-Wittgenstein,Bad Laasphe,05970028,57334,Bad Laasphe,2020-04-16,2009-12-18,472,35,0,Hausdach,0,0,2495,5.324,4.300,0.0,688.0,4.30,1450.0,44.0,853.0,802.0,1.0,

### Nicht benötigte Spalten sowie Spalten mit vielen fehlenden Werten löschen

In [8]:
# Welche Spalten weisen Nan-Values auf? 
nan_count = pv_raw.isna().sum()
pd.set_option('display.max_rows', 100)
print(nan_count )

EinheitMastrNummer                                0
DatumLetzteAktualisierung                         0
LokationMaStRNummer                           60857
NetzbetreiberpruefungStatus                       0
AnlagenbetreiberMastrNummer                    4300
Land                                              0
Bundesland                                       25
Landkreis                                        25
Gemeinde                                         25
Gemeindeschluessel                               25
Postleitzahl                                      2
Ort                                               2
Registrierungsdatum                               0
Inbetriebnahmedatum                           56666
EinheitSystemstatus                               0
EinheitBetriebsstatus                             0
NichtVorhandenInMigriertenEinheiten               0
NameStromerzeugungseinheit                        9
Weic_nv                                           0
Kraftwerksnu

In [9]:
# nicht benötigte Spalten sowie Spalten mit sehr vielen NaN-Werten löschen
drop_cols = ["EinheitMastrNummer", "LokationMaStRNummer", "Nebenausrichtung", "NebenausrichtungNeigungswinkel", 
             "Strasse", "StrasseNichtGefunden", "Hausnummer",
             "Hausnummer_nv", "HausnummerNichtGefunden", "Laengengrad", "Breitengrad", "Gemarkung", 
             "FlurFlurstuecknummern",
             "DatumDesBetreiberwechsels", "DatumRegistrierungDesBetreiberwechsels", 
             "AnschlussAnHoechstOderHochSpannung",
             "FernsteuerbarkeitDv", "FernsteuerbarkeitDr", "Adresszusatz", "GeplantesInbetriebnahmedatum", 
             "InAnspruchGenommeneFlaeche", "ArtDerFlaecheIds", 
             "InAnspruchGenommeneAckerflaeche", "DatumBeginnVoruebergehendeStilllegung", "Einsatzverantwortlicher", 
             "GenMastrNummer", "DatumWiederaufnahmeBetrieb", "Kraftwerksnummer", "WeicDisplayName", "Weic", 
             "EegMaStRNummer", "NetzbetreiberpruefungDatum", "Buergerenergie"]
pv_raw = pv_raw.drop(drop_cols, axis=1)
pv_raw

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Land,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitSystemstatus,EinheitBetriebsstatus,NichtVorhandenInMigriertenEinheiten,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Energietraeger,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,DatumEndgueltigeStilllegung
0,2020-02-20T16:28:35.2508122,1,ABR949444220202,84,1409.0,Münster,Münster,05515000,48147,Münster,2019-02-01,2007-07-20,472,35,0,Photovoltaikanlage ERWin4,0,0,2495,3.960,3.960,0.0,688.0,4.00,1450.0,22.0,853.0,802.0,1.0,699.0,809.0,713.0,None
1,2021-07-15T07:54:37.8637095,1,ABR930055871044,84,1402.0,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,2013-01-31,472,35,0,man-power,0,0,2495,7.410,7.410,0.0,689.0,8.30,1450.0,38.0,853.0,803.0,1.0,699.0,809.0,713.0,None
2,2019-05-28T08:26:38.0316692,1,ABR983885950021,84,1400.0,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,2016-02-19,472,35,0,eigene PVA,0,0,2495,5.040,5.000,0.0,689.0,5.00,1449.0,16.0,853.0,803.0,1.0,699.0,809.0,713.0,None
3,2020-12-07T08:19:07.9860793,1,ABR980052996196,84,1403.0,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,2016-12-16,472,35,0,Photovoltaik EFH Löbert - Mattinger Weg 6,0,0,2495,6.360,6.000,0.0,689.0,6.00,1448.0,24.0,853.0,1535.0,1.0,700.0,809.0,713.0,None
4,2021-01-25T11:18:30.0433603,1,ABR970373690446,84,1412.0,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,2011-12-08,472,35,0,"PV-Anlage 7,2 (Voll)",0,0,2495,7.200,7.200,0.0,689.0,7.60,1450.0,30.0,853.0,802.0,0.0,701.0,809.0,713.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3180108,2020-07-14T07:21:05.4509560,1,ABR941126007873,84,1409.0,Siegen-Wittgenstein,Bad Laasphe,05970028,57334,Bad Laasphe,2020-04-16,2009-12-18,472,35,0,Hausdach,0,0,2495,5.324,4.300,0.0,688.0,4.30,1450.0,44.0,853.0,802.0,1.0,697.0,810.0,713.0,None
3180109,2020-04-29T09:37:03.4024856,0,ABR903375397990,84,1409.0,Hochsauerlandkreis,Bestwig,05958008,59909,Bestwig,2020-04-29,2010-07-01,472,35,0,Scheune Hof,0,0,2495,16.215,16.215,0.0,688.0,16.50,1450.0,69.0,853.0,802.0,1.0,699.0,808.0,716.0,None
3180110,2020-11-26T08:57:11.0060494,1,ABR985070432688,84,1403.0,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,92699,Bechtsrieth,2020-04-29,2020-04-28,472,35,0,Familie Koller,0,0,2495,29.900,29.900,0.0,689.0,50.00,1450.0,90.0,853.0,803.0,0.0,701.0,809.0,713.0,None
3180111,2020-05-02T10:16:32.7818459,1,ABR925833676461,84,1405.0,Groß-Gerau,Riedstadt,06433011,64560,Riedstadt,2020-04-29,2016-05-31,472,35,0,Hausdach,0,0,2495,3.570,3.000,0.0,689.0,3.00,1449.0,14.0,853.0,803.0,1.0,NaN,809.0,713.0,None


### Endgültig stillgelegte PV-Anlagen aus dem Datensatz löschen

In [10]:
# Count endgültig stillgelegte PV-Analgen
(pv_raw.DatumEndgueltigeStilllegung).notnull().sum()

7078

In [11]:
# Design-Entscheidung: 7078 endgültig stillgelegte PV-Anlagen aus dem Datensatz löschen
pv_raw = pv_raw[(pv_raw.DatumEndgueltigeStilllegung).isnull()]
pv_raw

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Land,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitSystemstatus,EinheitBetriebsstatus,NichtVorhandenInMigriertenEinheiten,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Energietraeger,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,DatumEndgueltigeStilllegung
0,2020-02-20T16:28:35.2508122,1,ABR949444220202,84,1409.0,Münster,Münster,05515000,48147,Münster,2019-02-01,2007-07-20,472,35,0,Photovoltaikanlage ERWin4,0,0,2495,3.960,3.960,0.0,688.0,4.00,1450.0,22.0,853.0,802.0,1.0,699.0,809.0,713.0,None
1,2021-07-15T07:54:37.8637095,1,ABR930055871044,84,1402.0,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,2013-01-31,472,35,0,man-power,0,0,2495,7.410,7.410,0.0,689.0,8.30,1450.0,38.0,853.0,803.0,1.0,699.0,809.0,713.0,None
2,2019-05-28T08:26:38.0316692,1,ABR983885950021,84,1400.0,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,2016-02-19,472,35,0,eigene PVA,0,0,2495,5.040,5.000,0.0,689.0,5.00,1449.0,16.0,853.0,803.0,1.0,699.0,809.0,713.0,None
3,2020-12-07T08:19:07.9860793,1,ABR980052996196,84,1403.0,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,2016-12-16,472,35,0,Photovoltaik EFH Löbert - Mattinger Weg 6,0,0,2495,6.360,6.000,0.0,689.0,6.00,1448.0,24.0,853.0,1535.0,1.0,700.0,809.0,713.0,None
4,2021-01-25T11:18:30.0433603,1,ABR970373690446,84,1412.0,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,2011-12-08,472,35,0,"PV-Anlage 7,2 (Voll)",0,0,2495,7.200,7.200,0.0,689.0,7.60,1450.0,30.0,853.0,802.0,0.0,701.0,809.0,713.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3180108,2020-07-14T07:21:05.4509560,1,ABR941126007873,84,1409.0,Siegen-Wittgenstein,Bad Laasphe,05970028,57334,Bad Laasphe,2020-04-16,2009-12-18,472,35,0,Hausdach,0,0,2495,5.324,4.300,0.0,688.0,4.30,1450.0,44.0,853.0,802.0,1.0,697.0,810.0,713.0,None
3180109,2020-04-29T09:37:03.4024856,0,ABR903375397990,84,1409.0,Hochsauerlandkreis,Bestwig,05958008,59909,Bestwig,2020-04-29,2010-07-01,472,35,0,Scheune Hof,0,0,2495,16.215,16.215,0.0,688.0,16.50,1450.0,69.0,853.0,802.0,1.0,699.0,808.0,716.0,None
3180110,2020-11-26T08:57:11.0060494,1,ABR985070432688,84,1403.0,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,92699,Bechtsrieth,2020-04-29,2020-04-28,472,35,0,Familie Koller,0,0,2495,29.900,29.900,0.0,689.0,50.00,1450.0,90.0,853.0,803.0,0.0,701.0,809.0,713.0,None
3180111,2020-05-02T10:16:32.7818459,1,ABR925833676461,84,1405.0,Groß-Gerau,Riedstadt,06433011,64560,Riedstadt,2020-04-29,2016-05-31,472,35,0,Hausdach,0,0,2495,3.570,3.000,0.0,689.0,3.00,1449.0,14.0,853.0,803.0,1.0,NaN,809.0,713.0,None


### PV-Anlagen außerhalb Deutschlands löschen

In [12]:
# Sind alle aufgeführten PV-Anlagen in Deutschland? (Ländercode: 84)
pv_raw.groupby('Land').count()

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitSystemstatus,EinheitBetriebsstatus,NichtVorhandenInMigriertenEinheiten,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Energietraeger,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,DatumEndgueltigeStilllegung
Land,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
56,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
66,1,1,1,0,0,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
84,3173012,3173012,3173012,3173010,3173010,3173010,3173010,3173010,3173010,3173012,3116351,3173012,3173012,3173012,3173003,3173012,3173012,3173012,3173012,3173012,3127433,3147109,3169711,3097497,3088359,3172955,3087724,3108732,3123926,3098863,2764810,0
90,7,7,7,0,0,0,0,7,7,7,6,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,1,0
95,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
106,6,6,6,0,0,0,0,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,5,0
113,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,0,0,0,0
198,2,2,2,0,0,0,0,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,1,2,0,0
206,4,4,4,0,0,0,0,4,4,4,2,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,0


**23 PV-Anlagen befinden sich außerrhalb Deutschlands**

In [13]:
katalog_wert.loc[katalog_wert["Id"].isin([56., 66.,  84.,  90., 95., 106., 113., 198., 206.])]

,Id,Wert,KatalogKategorieId
48,56,Armenien,6
58,66,Belgien,6
76,84,Deutschland,6
82,90,Dänemark,6
87,95,Dominica,6
98,106,Frankreich,6
105,113,Ghana,6
190,198,Niederlande,6
198,206,Österreich,6


In [14]:
# Einträge außerhalb Deutschlands löschen
pv_raw = pv_raw[(pv_raw["Land"] == 84)]
pv_raw

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Land,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitSystemstatus,EinheitBetriebsstatus,NichtVorhandenInMigriertenEinheiten,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Energietraeger,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,DatumEndgueltigeStilllegung
0,2020-02-20T16:28:35.2508122,1,ABR949444220202,84,1409.0,Münster,Münster,05515000,48147,Münster,2019-02-01,2007-07-20,472,35,0,Photovoltaikanlage ERWin4,0,0,2495,3.960,3.960,0.0,688.0,4.00,1450.0,22.0,853.0,802.0,1.0,699.0,809.0,713.0,None
1,2021-07-15T07:54:37.8637095,1,ABR930055871044,84,1402.0,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,2013-01-31,472,35,0,man-power,0,0,2495,7.410,7.410,0.0,689.0,8.30,1450.0,38.0,853.0,803.0,1.0,699.0,809.0,713.0,None
2,2019-05-28T08:26:38.0316692,1,ABR983885950021,84,1400.0,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,2016-02-19,472,35,0,eigene PVA,0,0,2495,5.040,5.000,0.0,689.0,5.00,1449.0,16.0,853.0,803.0,1.0,699.0,809.0,713.0,None
3,2020-12-07T08:19:07.9860793,1,ABR980052996196,84,1403.0,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,2016-12-16,472,35,0,Photovoltaik EFH Löbert - Mattinger Weg 6,0,0,2495,6.360,6.000,0.0,689.0,6.00,1448.0,24.0,853.0,1535.0,1.0,700.0,809.0,713.0,None
4,2021-01-25T11:18:30.0433603,1,ABR970373690446,84,1412.0,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,2011-12-08,472,35,0,"PV-Anlage 7,2 (Voll)",0,0,2495,7.200,7.200,0.0,689.0,7.60,1450.0,30.0,853.0,802.0,0.0,701.0,809.0,713.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3180108,2020-07-14T07:21:05.4509560,1,ABR941126007873,84,1409.0,Siegen-Wittgenstein,Bad Laasphe,05970028,57334,Bad Laasphe,2020-04-16,2009-12-18,472,35,0,Hausdach,0,0,2495,5.324,4.300,0.0,688.0,4.30,1450.0,44.0,853.0,802.0,1.0,697.0,810.0,713.0,None
3180109,2020-04-29T09:37:03.4024856,0,ABR903375397990,84,1409.0,Hochsauerlandkreis,Bestwig,05958008,59909,Bestwig,2020-04-29,2010-07-01,472,35,0,Scheune Hof,0,0,2495,16.215,16.215,0.0,688.0,16.50,1450.0,69.0,853.0,802.0,1.0,699.0,808.0,716.0,None
3180110,2020-11-26T08:57:11.0060494,1,ABR985070432688,84,1403.0,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,92699,Bechtsrieth,2020-04-29,2020-04-28,472,35,0,Familie Koller,0,0,2495,29.900,29.900,0.0,689.0,50.00,1450.0,90.0,853.0,803.0,0.0,701.0,809.0,713.0,None
3180111,2020-05-02T10:16:32.7818459,1,ABR925833676461,84,1405.0,Groß-Gerau,Riedstadt,06433011,64560,Riedstadt,2020-04-29,2016-05-31,472,35,0,Hausdach,0,0,2495,3.570,3.000,0.0,689.0,3.00,1449.0,14.0,853.0,803.0,1.0,NaN,809.0,713.0,None


### Zuordnung der Katalogwerte zu den entsprechenden Katalogkategorien

#### Bundesland

In [15]:
pv_raw['Bundesland'].unique()

array([1409., 1402., 1400., 1403., 1412., 1408., 1410., 1407., 1414.,
       1411., 1413., 1405., 1415., 1406., 1401., 1404.,   nan, 1416.])

In [16]:
katalog_wert.loc[katalog_wert["Id"].isin([1409., 1402., 1400., 1403., 1412., 1408., 1410., 1407., 1414.,
       1411., 1413., 1405., 1415., 1406., 1401., 1404., 1416.])]

,Id,Wert,KatalogKategorieId
921,1400,Brandenburg,101
922,1401,Berlin,101
923,1402,Baden-Württemberg,101
924,1403,Bayern,101
925,1404,Bremen,101
926,1405,Hessen,101
927,1406,Hamburg,101
928,1407,Mecklenburg-Vorpommern,101
929,1408,Niedersachsen,101
930,1409,Nordrhein-Westfalen,101


Ausschließliche Wirtschaftszone: Nord- und Ostsee (Fläche: 32.982 km²)  
Etwa 70 % der Meeresfläche des gesamten deutschen Nordseegebiets und 29 % der Meeresfläche des gesamten deutschen Ostseegebiets.

#### Lage

In [17]:
pv_raw['Lage'].unique()

array([ 853., 2484.,  852.,   nan, 2961., 3002., 3058.])

In [18]:
katalog_wert.loc[katalog_wert["Id"].isin([853., 2484.,  852., 2961., 3002., 3058.])]

,Id,Wert,KatalogKategorieId
569,852,Freifläche,82
570,853,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",82
1253,2484,Bauliche Anlagen (Sonstige),82
1595,2961,Steckerfertige Erzeugungsanlage (sog. Plug-In-...,82
1607,3002,Gewässer,82
1619,3058,Großparkplatz,82


#### EinheitBetriebsstatus

In [19]:
pv_raw['EinheitBetriebsstatus'].unique()

array([35, 31, 37, 38], dtype=int64)

In [20]:
katalog_wert.loc[katalog_wert["Id"].isin([35, 31, 37, 38])]

,Id,Wert,KatalogKategorieId
27,31,In Planung,4
28,35,In Betrieb,4
29,37,Vorübergehend stillgelegt,4
30,38,Endgültig stillgelegt,4


#### EinheitSystemstatus

In [21]:
pv_raw['EinheitSystemstatus'].unique()

array([472], dtype=int64)

In [22]:
katalog_wert.loc[katalog_wert["Id"].isin([472.])]

,Id,Wert,KatalogKategorieId
357,472,Aktiviert,19


#### Energietraeger

In [23]:
pv_raw['Energietraeger'].unique()

array([2495], dtype=int64)

In [24]:
katalog_wert.loc[katalog_wert["Id"].isin([2495.])]

,Id,Wert,KatalogKategorieId
1264,2495,Solare Strahlungsenergie,148


#### Einspeisungsart

In [25]:
pv_raw['Einspeisungsart'].unique()

array([688., 689.,  nan])

In [26]:
katalog_wert.loc[katalog_wert["Id"].isin([688., 689.])]

,Id,Wert,KatalogKategorieId
445,688,Volleinspeisung,52
446,689,Teileinspeisung (einschließlich Eigenverbrauch),52


#### GemeinsamerWechselrichterMitSpeicher

In [27]:
pv_raw['GemeinsamerWechselrichterMitSpeicher'].unique()

array([1450., 1449., 1448.,   nan])

In [28]:
katalog_wert.loc[katalog_wert["Id"].isin([1450., 1449., 1448.])]

,Id,Wert,KatalogKategorieId
952,1448,Stromspeicher vorhanden - gemeinsamer Wechselr...,105
953,1449,Stromspeicher vorhanden - kein gemeinsamer Wec...,105
954,1450,Kein Stromspeicher vorhanden,105


#### Leistungsbegrenzung

In [29]:
pv_raw['Leistungsbegrenzung'].unique()

array([ 802.,  803., 1535.,  804.,   nan,  805.])

In [30]:
katalog_wert.loc[katalog_wert["Id"].isin([802.,  803., 1535.,  804., 805.])]

,Id,Wert,KatalogKategorieId
523,802,Nein,76
524,803,"Ja, auf 70%",76
525,804,"Ja, auf 60%",76
526,805,"Ja, auf 50%",76
1039,1535,"Ja, sonstige",76


#### Hauptausrichtung

In [31]:
pv_raw['Hauptausrichtung'].unique()

array([699., 700., 701.,  nan, 697., 698., 696., 695., 702., 703., 704.])

In [32]:
katalog_wert.loc[katalog_wert["Id"].isin([699., 700., 701., 697., 698., 696., 695., 702., 703., 704.])]

,Id,Wert,KatalogKategorieId
451,695,Nord,34
452,696,Nord-Ost,34
453,697,Ost,34
454,698,Süd-Ost,34
455,699,Süd,34
456,700,Süd-West,34
457,701,West,34
458,702,Nord-West,34
459,703,nachgeführt,34
460,704,Ost-West,34


#### HauptausrichtungNeigungswinkel

In [33]:
pv_raw['HauptausrichtungNeigungswinkel'].unique()

array([809., 810.,  nan, 808., 807., 811., 806.])

In [34]:
katalog_wert.loc[katalog_wert["Id"].isin([809., 810., 808., 807., 811., 806.])]

,Id,Wert,KatalogKategorieId
527,806,Fassadenintegriert,33
528,807,> 60 Grad,33
529,808,40 - 60 Grad,33
530,809,20 - 40 Grad,33
531,810,< 20 Grad,33
532,811,Nachgeführt,33


#### Nutzungsbereich

In [35]:
pv_raw['Nutzungsbereich'].unique()

array([713., 716., 714.,  nan, 718., 715., 717.])

In [36]:
katalog_wert.loc[katalog_wert["Id"].isin([713., 716., 714., 718., 715., 717.])]

,Id,Wert,KatalogKategorieId
465,713,Haushalt,57
466,714,"Gewerbe, Handel und Dienstleistungen",57
467,715,Industrie,57
468,716,Landwirtschaft,57
469,717,Öffentliches Gebäude,57
470,718,Sonstige,57


### Nicht aussagekäftige Spalten mit nur einer Ausprägung löschen

In [37]:
# wieviele Ausprägungen gibt es in jeder Spalte?
uniqueValues = pv_raw.nunique()
print(uniqueValues)

DatumLetzteAktualisierung                   3098385
NetzbetreiberpruefungStatus                       2
AnlagenbetreiberMastrNummer                 2606935
Land                                              1
Bundesland                                       17
Landkreis                                       382
Gemeinde                                      10563
Gemeindeschluessel                            11107
Postleitzahl                                   8182
Ort                                           11480
Registrierungsdatum                            1613
Inbetriebnahmedatum                           11345
EinheitSystemstatus                               1
EinheitBetriebsstatus                             4
NichtVorhandenInMigriertenEinheiten               1
NameStromerzeugungseinheit                  1393077
Weic_nv                                           2
Kraftwerksnummer_nv                               2
Energietraeger                                    1
Bruttoleistu

In [38]:
# Löschen von Spalten, die nur eine/keine Ausprägung haben, da diese keinen Mehrwert liefern
drop_cols = ["DatumEndgueltigeStilllegung", "Energietraeger", "NichtVorhandenInMigriertenEinheiten", "EinheitSystemstatus", 
             "Land"]
pv_raw = pv_raw.drop(drop_cols, axis=1)
pv_raw

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich
0,2020-02-20T16:28:35.2508122,1,ABR949444220202,1409.0,Münster,Münster,05515000,48147,Münster,2019-02-01,2007-07-20,35,Photovoltaikanlage ERWin4,0,0,3.960,3.960,0.0,688.0,4.00,1450.0,22.0,853.0,802.0,1.0,699.0,809.0,713.0
1,2021-07-15T07:54:37.8637095,1,ABR930055871044,1402.0,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,2013-01-31,35,man-power,0,0,7.410,7.410,0.0,689.0,8.30,1450.0,38.0,853.0,803.0,1.0,699.0,809.0,713.0
2,2019-05-28T08:26:38.0316692,1,ABR983885950021,1400.0,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,2016-02-19,35,eigene PVA,0,0,5.040,5.000,0.0,689.0,5.00,1449.0,16.0,853.0,803.0,1.0,699.0,809.0,713.0
3,2020-12-07T08:19:07.9860793,1,ABR980052996196,1403.0,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,2016-12-16,35,Photovoltaik EFH Löbert - Mattinger Weg 6,0,0,6.360,6.000,0.0,689.0,6.00,1448.0,24.0,853.0,1535.0,1.0,700.0,809.0,713.0
4,2021-01-25T11:18:30.0433603,1,ABR970373690446,1412.0,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,2011-12-08,35,"PV-Anlage 7,2 (Voll)",0,0,7.200,7.200,0.0,689.0,7.60,1450.0,30.0,853.0,802.0,0.0,701.0,809.0,713.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3180108,2020-07-14T07:21:05.4509560,1,ABR941126007873,1409.0,Siegen-Wittgenstein,Bad Laasphe,05970028,57334,Bad Laasphe,2020-04-16,2009-12-18,35,Hausdach,0,0,5.324,4.300,0.0,688.0,4.30,1450.0,44.0,853.0,802.0,1.0,697.0,810.0,713.0
3180109,2020-04-29T09:37:03.4024856,0,ABR903375397990,1409.0,Hochsauerlandkreis,Bestwig,05958008,59909,Bestwig,2020-04-29,2010-07-01,35,Scheune Hof,0,0,16.215,16.215,0.0,688.0,16.50,1450.0,69.0,853.0,802.0,1.0,699.0,808.0,716.0
3180110,2020-11-26T08:57:11.0060494,1,ABR985070432688,1403.0,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,92699,Bechtsrieth,2020-04-29,2020-04-28,35,Familie Koller,0,0,29.900,29.900,0.0,689.0,50.00,1450.0,90.0,853.0,803.0,0.0,701.0,809.0,713.0
3180111,2020-05-02T10:16:32.7818459,1,ABR925833676461,1405.0,Groß-Gerau,Riedstadt,06433011,64560,Riedstadt,2020-04-29,2016-05-31,35,Hausdach,0,0,3.570,3.000,0.0,689.0,3.00,1449.0,14.0,853.0,803.0,1.0,NaN,809.0,713.0


In [39]:
# pv_raw.info()

In [40]:
# Welche Spalten weisen noch immer Nan-Values auf? 
nan_count = pv_raw.isna().sum()
pd.set_option('display.max_rows', 100)
print(nan_count )

DatumLetzteAktualisierung                        0
NetzbetreiberpruefungStatus                      0
AnlagenbetreiberMastrNummer                      0
Bundesland                                       2
Landkreis                                        2
Gemeinde                                         2
Gemeindeschluessel                               2
Postleitzahl                                     2
Ort                                              2
Registrierungsdatum                              0
Inbetriebnahmedatum                          56661
EinheitBetriebsstatus                            0
NameStromerzeugungseinheit                       9
Weic_nv                                          0
Kraftwerksnummer_nv                              0
Bruttoleistung                                   0
Nettonennleistung                                0
FernsteuerbarkeitNb                          45579
Einspeisungsart                              25903
ZugeordneteWirkleistungWechselr

Der bereinigte Datensatz enthält 3.173.012 Zeilen und 28 Spalten.  
Es sind jedoch noch zahlreiche fehlende Werte enthalten.

### Data Cleaning Standort-Informationen

#### Datensätze mit fehlenden Standort-Daten löschen

In [41]:
# Standort-Spalten mit fehlenden Werten
missings = ['Bundesland', 'Landkreis', 'Gemeinde', 'Gemeindeschluessel', 'Postleitzahl', 'Ort']
missing_standort = pv_raw[pv_raw[missings].isna().any(axis=1)]
print(missing_standort[missings])

         Bundesland Landkreis Gemeinde Gemeindeschluessel Postleitzahl   Ort
92660           NaN      None     None               None         None  None
1145934         NaN      None     None               None         None  None


In [42]:
# die fehlenden Werte in den Spalten Bundesland, Landkreis, Gemeinde, Postleitzahl und Ort 
# beziehen sich auf lediglich zwei PV-Anlagen (Index 92660 und 1145934) und werden daher gelöscht
pv_raw = pv_raw.drop([92660, 1145934])
pv_raw

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich
0,2020-02-20T16:28:35.2508122,1,ABR949444220202,1409.0,Münster,Münster,05515000,48147,Münster,2019-02-01,2007-07-20,35,Photovoltaikanlage ERWin4,0,0,3.960,3.960,0.0,688.0,4.00,1450.0,22.0,853.0,802.0,1.0,699.0,809.0,713.0
1,2021-07-15T07:54:37.8637095,1,ABR930055871044,1402.0,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,2013-01-31,35,man-power,0,0,7.410,7.410,0.0,689.0,8.30,1450.0,38.0,853.0,803.0,1.0,699.0,809.0,713.0
2,2019-05-28T08:26:38.0316692,1,ABR983885950021,1400.0,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,2016-02-19,35,eigene PVA,0,0,5.040,5.000,0.0,689.0,5.00,1449.0,16.0,853.0,803.0,1.0,699.0,809.0,713.0
3,2020-12-07T08:19:07.9860793,1,ABR980052996196,1403.0,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,2016-12-16,35,Photovoltaik EFH Löbert - Mattinger Weg 6,0,0,6.360,6.000,0.0,689.0,6.00,1448.0,24.0,853.0,1535.0,1.0,700.0,809.0,713.0
4,2021-01-25T11:18:30.0433603,1,ABR970373690446,1412.0,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,2011-12-08,35,"PV-Anlage 7,2 (Voll)",0,0,7.200,7.200,0.0,689.0,7.60,1450.0,30.0,853.0,802.0,0.0,701.0,809.0,713.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3180108,2020-07-14T07:21:05.4509560,1,ABR941126007873,1409.0,Siegen-Wittgenstein,Bad Laasphe,05970028,57334,Bad Laasphe,2020-04-16,2009-12-18,35,Hausdach,0,0,5.324,4.300,0.0,688.0,4.30,1450.0,44.0,853.0,802.0,1.0,697.0,810.0,713.0
3180109,2020-04-29T09:37:03.4024856,0,ABR903375397990,1409.0,Hochsauerlandkreis,Bestwig,05958008,59909,Bestwig,2020-04-29,2010-07-01,35,Scheune Hof,0,0,16.215,16.215,0.0,688.0,16.50,1450.0,69.0,853.0,802.0,1.0,699.0,808.0,716.0
3180110,2020-11-26T08:57:11.0060494,1,ABR985070432688,1403.0,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,92699,Bechtsrieth,2020-04-29,2020-04-28,35,Familie Koller,0,0,29.900,29.900,0.0,689.0,50.00,1450.0,90.0,853.0,803.0,0.0,701.0,809.0,713.0
3180111,2020-05-02T10:16:32.7818459,1,ABR925833676461,1405.0,Groß-Gerau,Riedstadt,06433011,64560,Riedstadt,2020-04-29,2016-05-31,35,Hausdach,0,0,3.570,3.000,0.0,689.0,3.00,1449.0,14.0,853.0,803.0,1.0,NaN,809.0,713.0


In [43]:
# Welche Spalten weisen noch immer Nan-Values auf? 
nan_count = pv_raw.isna().sum()
print(nan_count )

DatumLetzteAktualisierung                        0
NetzbetreiberpruefungStatus                      0
AnlagenbetreiberMastrNummer                      0
Bundesland                                       0
Landkreis                                        0
Gemeinde                                         0
Gemeindeschluessel                               0
Postleitzahl                                     0
Ort                                              0
Registrierungsdatum                              0
Inbetriebnahmedatum                          56661
EinheitBetriebsstatus                            0
NameStromerzeugungseinheit                       9
Weic_nv                                          0
Kraftwerksnummer_nv                              0
Bruttoleistung                                   0
Nettonennleistung                                0
FernsteuerbarkeitNb                          45579
Einspeisungsart                              25903
ZugeordneteWirkleistungWechselr

#### Standardisieren der Postleitzahlen

Im Datensatz sind **8182** individuelle Postleitzahlen (PLZ) enthalten.  
Laut PLZ-Verzeichnis gibt es jedoch nur **8170** Postleitzahlen.    
Quelle: Source: https://www.suche-postleitzahl.org/plz-karte-erstellen  (zuletzt aktualisiert am 15.07.2023)  
Da die PLZ später mit weiteren Datenquellen gematcht werden soll, müssen die Postleitzahlen an das Standardschema angepasst werden.    
Dazu wird der pv_raw DataFrame mit der PLZ-Liste aus oben genannter Quelle abgeglichen.

In [44]:
# PLZ File einlesen
plz_de = gpd.read_file("../input/Data/plz-5stellig-centroid.shp", dtype={"plz": str})
plz_de

,plz,note,einwohner,qkm,geometry
0,64743,"Situation unklar, evtl. haben die HÃ¤user Marb...",3,0.082066,POINT (8.98571 49.60635)
1,81248,81248 MÃ¼nchen,121,1.984763,POINT (11.40231 48.14781)
2,60315,60315 Frankfurt am Main (FOUR),0,0.017285,POINT (8.67384 50.11224)
3,99331,99331 Geratal,4523,20.207080,POINT (10.83066 50.72324)
4,60312,60312 Frankfurt am Main (Omniturm),0,0.001829,POINT (8.67302 50.11155)
...,...,...,...,...,...
8165,63874,63874 Dammbach,1819,32.868150,POINT (9.34341 49.85922)
8166,63856,63856 Bessenbach,5844,29.855783,POINT (9.25702 49.95488)
8167,63869,63869 HeigenbrÃ¼cken,2208,8.415089,POINT (9.37251 50.02722)
8168,75339,75339 HÃ¶fen an der Enz,1619,9.081549,POINT (8.57697 48.80192)


In [45]:
# Postleitzahlen auslesen
plz = plz_de[["plz"]]
plz = plz.rename(columns={"plz": "Postleitzahl"}).astype(str)
plz

,Postleitzahl
0,64743
1,81248
2,60315
3,99331
4,60312
...,...
8165,63874
8166,63856
8167,63869
8168,75339


In [46]:
pv_raw_plz = pv_raw.copy()
#pv_raw_plz

In [47]:
df_all = pv_raw_plz.merge(plz.drop_duplicates(), on=['Postleitzahl'], 
                   how='left', indicator=True)

no_match = df_all[(df_all._merge != 'both')]
no_match

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,_merge
3699,2019-07-08T16:16:40.0135992,1,ABR971812852449,1415.0,Ilm-Kreis,Langewiesen,16070032,98704,Langewiesen,2019-02-01,2019-01-07,35,Photovoltaikanlage_2019,0,0,8.640,8.20,0.0,689.0,8.20,1450.0,27.0,853.0,803.0,1.0,698.0,809.0,713.0,left_only
11112,2019-05-28T08:26:38.0316692,1,ABR948103048676,1408.0,Hildesheim,Weenzen,03254040,31096,Weenzen,2019-02-03,2018-06-01,35,PV_1_MS_OS_2018,0,0,9.990,9.00,0.0,689.0,9.00,1450.0,37.0,853.0,803.0,1.0,700.0,809.0,713.0,left_only
20092,2020-10-29T05:34:01.2139449,1,ABR981873439435,1408.0,Hildesheim,Sehlem,03254030,31196,Sehlem,2019-02-06,2009-11-26,35,PV-Sehlem,0,0,8.290,8.00,0.0,688.0,8.00,1450.0,36.0,853.0,802.0,1.0,698.0,808.0,714.0,left_only
25045,2019-07-12T12:45:46.9941940,0,ABR947053125413,1415.0,Ilm-Kreis,Wolfsberg,16070055,98704,Wolfsberg,2019-02-08,2019-02-08,35,Bischoff1,0,0,5.360,5.36,0.0,689.0,6.00,1450.0,16.0,853.0,802.0,1.0,699.0,808.0,713.0,left_only
34642,2020-01-04T10:27:53.7453173,0,ABR995956971316,1415.0,Ilm-Kreis,Schmiedefeld am Rennsteig,16070046,98711,Schmiedefeld,2019-02-12,2011-12-27,35,Hausdach,0,0,33.000,29.15,0.0,688.0,29.15,NaN,264.0,853.0,802.0,0.0,698.0,809.0,713.0,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3126777,2020-06-02T03:03:14.6318021,1,ABR963429471410,1415.0,Kyffhäuserkreis,Gorsleben,16065022,06578,Oldisleben,2020-04-05,2011-11-29,35,PVA Sachsenburg TEN2,0,0,4.320,3.60,1.0,688.0,3.60,1450.0,18.0,853.0,802.0,1.0,698.0,810.0,714.0,left_only
3148013,2020-04-16T15:29:58.9978591,0,ABR924097511839,1415.0,Nordhausen,Herrmannsacker,16062018,99762,Herrmannsacker,2020-04-16,2015-06-30,35,PVA Herrmannsacker,0,0,5.760,5.76,1.0,688.0,5.76,1450.0,24.0,853.0,802.0,1.0,699.0,809.0,714.0,left_only
3157551,2020-04-21T15:45:11.5153019,1,ABR980334524963,1415.0,Sömmerda,Kannawurf,16068028,06578,Kannawurf,2020-04-21,2018-01-26,35,Scheune,0,0,9.990,9.99,0.0,689.0,10.00,1450.0,37.0,853.0,803.0,1.0,699.0,808.0,718.0,left_only
3161484,2021-12-16T10:41:59.6181623,0,ABR921412278378,1415.0,Kyffhäuserkreis,Oldisleben,16065054,06578,Oldisleben,2020-04-06,2011-04-19,35,PV-Anlage,0,0,2.585,1.80,0.0,689.0,1.80,1449.0,11.0,853.0,1535.0,1.0,699.0,809.0,713.0,left_only


Es gibt **198** Reihen im Datensatz, die Postleitzahlen beinhalten, die es laut Postleitzahlenverzeichnis nicht gibt.

In [48]:
no_match["Postleitzahl"].unique()

array(['98704', '31096', '31196', '98711', '98678', '31035', '31097',
       '98708', '06578', '31093', '98554', '21784', '98739', '37449',
       '83565', '07334', '64759', '31039', '25868', '31094', '99762',
       '39599'], dtype=object)

Alle ungültigen Postleitzahlen auf www.suche-postleitzahl.org prüfen und anhand Gemeinde und Landkreis die richtigen PLZ vergeben.   
Beispielcode: no_match[no_match["Postleitzahl"]==98704]

**Ergebnisse der PLZ Recherche:**  
98704 gibt es nicht, ersetzen durch 98693 (Langewiesen, Oehrenstock, Wolfsberg)   
31096 gibt es nicht, ersetzen durch 31089 (Hildesheim, Weenzen)   
31196 gibt es nicht, ersetzen durch 31089 (Hildesheim, Sehlem)    
98711 gibt es nicht, ersetzen durch 98528 (Schmiedefeld am Rennsteig)   
98678 gibt es nicht, ersetzen durch 98673 (Hildburghausen, Sachsenbrunn)    
31035 gibt es nicht, ersetzen durch 31028 (Hildesheim, Despetal)   
31097 gibt es nicht, ersetzen durch 31195 (Hildesheim, Harbarnsen)   
98708 gibt es nicht, ersetzen durch 98694 (Hildesheim, Gehren, Pennewitz)   
6578  gibt es nicht, ersetzen durch 6577 (für Landkreis Kyffhäuserkreis)   
                    und ersetzen durch 99638 (für Landkreis Sömmerda)   
31093 gibt es nicht, ersetzen durch 31089 (Hildesheim, Hoyershausen)   
98554 gibt es nicht, ersetzen durch 98544 (Schmalkalden-Meiningen, Benshausen)    
21784 gibt es nicht, ersetzen durch 21781 (Cuxhaven, Geversdorf)  
98739 gibt es nicht, ersetzen durch 07318 (für Gemeinde Schmiedefeld)   
                    und ersetzen durch 98724 (für Gemeinde Lichte)   
37449 gibt es nicht, ersetzen durch 37445 (Osterode am Harz, Zorge)  
83565 gibt es nicht, ersetzen durch 83553 (Ebersberg, Frauenneuharting)   
7334  gibt es nicht, ersetzen durch 07333 (Saalfeld-Rudolstadt, Kamsdorf)  
64759 gibt es nicht, ersetzen durch 64760 (Odenwaldkreis, Sensbachtal)   
31039 gibt es nicht, ersetzen durch 31028 (Hildesheim, Rheden)   
25868 gibt es nicht, ersetzen durch 25879 (Schleswig-Flensburg, Norderstapel)   
31094 gibt es nicht, ersetzen durch 31089 (Hildesheim, Marienhagen)   
99762 gibt es nicht, ersetzen durch 99768 (Herrmannsacker, Neustadt/Harz)     
39599 gibt es nicht, ersetzen durch 39629 (Stendal, Bismark (Altmark))   

In [49]:
plz_dict=  {'98704'  : '98693', 
            '31096'  : '31089',
            '31196'  : '31089',
            '98711'  : '98528',
            '98678'  : '98673',
            '31035'  : '31028',
            '31097'  : '31195',
            '98708'  : '98694',
            '31093'  : '31089',
            '98554'  : '98544',
            '21784'  : '21781',  
            '37449'  : '37445',
            '83565'  : '83553',
            '07334'  : '07333',
            '64759'  : '64760',
            '31039'  : '31028',
            '25868'  : '25879',
            '31094'  : '31089',
            '39599'  : '39629',
            '99762'  : '99768'}

In [50]:
# Austausch der eindeutig zuzuordnenden PLZ
pv_raw_plz = pv_raw_plz.replace({"Postleitzahl": plz_dict})
#pv_raw_plz

In [51]:
# Austausch Postleitzahl 06578
pv_raw_plz['Postleitzahl'] = np.where(
   (pv_raw_plz['Landkreis'] == 'Kyffhäuserkreis') & (pv_raw_plz['Postleitzahl'] == '06578') , '06577', pv_raw_plz['Postleitzahl']
   )

pv_raw_plz['Postleitzahl'] = np.where(
   (pv_raw_plz['Landkreis'] == 'Sömmerda') & (pv_raw_plz['Postleitzahl'] == '06578') , '99638', pv_raw_plz['Postleitzahl']
   )

In [52]:
# Check, ob alle PLZ 06578 überschrieben wurden
pv_raw_plz[pv_raw_plz["Postleitzahl"]=='06578']

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich


In [53]:
# Austausch Postleitzahl 98739
pv_raw_plz['Postleitzahl'] = np.where(
   (pv_raw_plz['Gemeinde'] == 'Schmiedefeld') & (pv_raw_plz['Postleitzahl'] == '98739') , '07318', pv_raw_plz['Postleitzahl']
   )

pv_raw_plz['Postleitzahl'] = np.where(
   (pv_raw_plz['Gemeinde'] == 'Lichte') & (pv_raw_plz['Postleitzahl'] == '98739') , '98724', pv_raw_plz['Postleitzahl']
   )

In [54]:
# Check, ob alle PLZ 98739 überschrieben wurden
pv_raw_plz[pv_raw_plz["Postleitzahl"]=='98739']

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich


In [55]:
# pv_raw_plz

In [56]:
# wieviele Ausprägungen gibt es in jeder Spalte?
uniqueValues = pv_raw_plz.nunique()
print(uniqueValues)

DatumLetzteAktualisierung                   3098383
NetzbetreiberpruefungStatus                       2
AnlagenbetreiberMastrNummer                 2606935
Bundesland                                       17
Landkreis                                       382
Gemeinde                                      10563
Gemeindeschluessel                            11107
Postleitzahl                                   8160
Ort                                           11480
Registrierungsdatum                            1613
Inbetriebnahmedatum                           11345
EinheitBetriebsstatus                             4
NameStromerzeugungseinheit                  1393075
Weic_nv                                           2
Kraftwerksnummer_nv                               2
Bruttoleistung                                50500
Nettonennleistung                             34614
FernsteuerbarkeitNb                               2
Einspeisungsart                                   2
ZugeordneteW

Es gibt nicht zu allen Postleitzahlen Einträge im Marktstammdatenregister.   
Einige PLZ sind exklusiv für besondere Gebäude, wie Botschaften vergeben.

In [57]:
# Check, ob nur noch gültige PLZ im Datensatz enthalten sind
df_all = pv_raw_plz.merge(plz.drop_duplicates(), on=['Postleitzahl'], 
                   how='left', indicator=True)

no_match = df_all[(df_all._merge != 'both')]
pd.set_option('display.max_columns', 100)
no_match

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,_merge


#### Amtlicher Gemeindeschlüssel (AGS)
**Amtlicher Gemeindeschlüssel (AGS)**  
8-stelliger Schlüssel zur eindeutigen Identifizierung einer Gemeinde mit den Bestandteilen:  
Bundesland (2 Stellen), Regierungsbezirk (1 Stelle), Kreis (2 Stellen) und Gemeinde (3 Stellen).  
**Regionalschlüssel (RS)**  
12-stelliger Schlüssel zur eindeutigen Identifizierung einer Gemeinde mit den Bestandteilen: Bundesland (2 Stellen),   Regierungsbezirk (1 Stelle), Kreis (2 Stellen), Gemeindeverband (4 Stellen) und Gemeinde (3 Stellen).  

In [58]:
pv_raw_plz['len'] = pv_raw_plz['Gemeindeschluessel'].astype(str).apply(lambda x: len(x))
# Anzahl der Werte pro Länge zählen
anzahl_nach_laenge = pv_raw_plz['len'].value_counts().reset_index()
anzahl_nach_laenge.columns = ['Laenge', 'Anzahl_Werte']
print(anzahl_nach_laenge)

   Laenge  Anzahl_Werte
0       8       3173010


**Alle Gemeindeschlüssel weisen 8 Stellen auf und sind damit Amtliche Gemeindeschlüssel (AGS).**

#### Neue Tabelle: Gemeindeschlüssel Bundesländer, 2-stellig 
Ziffernfolgen für die Bundesländer (erste und zweite Ziffer des AGS)   
01 Schleswig-Holstein   
02 Hamburg   
03 Niedersachsen   
04 Bremen   
05 Nordrhein-Westfalen   
06 Hessen   
07 Rheinland-Pfalz   
08 Baden-Württemberg   
09 Bayern   
10 Saarland   
11 Berlin   
12 Brandenburg   
13 Mecklenburg-Vorpommern   
14 Sachsen  
15 Sachsen-Anhalt  
16 Thüringen  

In [59]:
bundesland = katalog_wert.loc[katalog_wert['Id'].isin([1409., 1402., 1400., 1403., 1412., 1408., 1410., 1407., 1414.,
       1411., 1413., 1405., 1415., 1406., 1401., 1404., 1416.])]
bundesland = bundesland.rename(columns={'Wert': 'Bundesland'})
# bundesland

In [60]:
Gemeindeschluessel = ['12', '11', '08', '09', '04', '06', '02', '13', '03', '05', '07', '01', '10', '14', '15', '16', None]
bundesland['Gemeindeschluessel'] = Gemeindeschluessel
bundesland

,Id,Bundesland,KatalogKategorieId,Gemeindeschluessel
921,1400,Brandenburg,101,12
922,1401,Berlin,101,11
923,1402,Baden-Württemberg,101,08
924,1403,Bayern,101,09
925,1404,Bremen,101,04
926,1405,Hessen,101,06
927,1406,Hamburg,101,02
928,1407,Mecklenburg-Vorpommern,101,13
929,1408,Niedersachsen,101,03
930,1409,Nordrhein-Westfalen,101,05


In [61]:
# bundesland als Pickle speichern
bundesland.to_pickle('bundesland.pkl')

#### Gemeindeschlüssel 8-stellig und 5-stellig

In [62]:
gemeindeschluessel = pd.DataFrame({'AGS-8': pv_raw_plz['Gemeindeschluessel'].unique()})
gemeindeschluessel

,AGS-8
0,05515000
1,08136065
2,12063208
3,09375180
4,10044115
...,...
11102,16063019
11103,07232038
11104,14729230
11105,16075069


In [63]:
# die ersten 5 Zeichen ds AGS-8 bilden den AGS-5
def erste_5_zeichen(text):
    return text[:5]
gemeindeschluessel['AGS-5'] = gemeindeschluessel['AGS-8'].apply(erste_5_zeichen)
gemeindeschluessel

,AGS-8,AGS-5
0,05515000,05515
1,08136065,08136
2,12063208,12063
3,09375180,09375
4,10044115,10044
...,...,...
11102,16063019,16063
11103,07232038,07232
11104,14729230,14729
11105,16075069,16075


In [64]:
gemeindeschluessel.nunique()

AGS-8    11107
AGS-5      403
dtype: int64

In [65]:
# Merge AGS 8-stellig und AGS 5-stellig mit der Marktstammdaten-Tabelle
pv_raw_plz.rename(columns={'Gemeindeschluessel': 'AGS-8'}, inplace=True)
pv_raw_plz = pv_raw_plz.merge(gemeindeschluessel, on='AGS-8', how='left')
pv_raw_plz

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,AGS-8,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,len,AGS-5
0,2020-02-20T16:28:35.2508122,1,ABR949444220202,1409.0,Münster,Münster,05515000,48147,Münster,2019-02-01,2007-07-20,35,Photovoltaikanlage ERWin4,0,0,3.960,3.960,0.0,688.0,4.00,1450.0,22.0,853.0,802.0,1.0,699.0,809.0,713.0,8,05515
1,2021-07-15T07:54:37.8637095,1,ABR930055871044,1402.0,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,2013-01-31,35,man-power,0,0,7.410,7.410,0.0,689.0,8.30,1450.0,38.0,853.0,803.0,1.0,699.0,809.0,713.0,8,08136
2,2019-05-28T08:26:38.0316692,1,ABR983885950021,1400.0,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,2016-02-19,35,eigene PVA,0,0,5.040,5.000,0.0,689.0,5.00,1449.0,16.0,853.0,803.0,1.0,699.0,809.0,713.0,8,12063
3,2020-12-07T08:19:07.9860793,1,ABR980052996196,1403.0,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,2016-12-16,35,Photovoltaik EFH Löbert - Mattinger Weg 6,0,0,6.360,6.000,0.0,689.0,6.00,1448.0,24.0,853.0,1535.0,1.0,700.0,809.0,713.0,8,09375
4,2021-01-25T11:18:30.0433603,1,ABR970373690446,1412.0,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,2011-12-08,35,"PV-Anlage 7,2 (Voll)",0,0,7.200,7.200,0.0,689.0,7.60,1450.0,30.0,853.0,802.0,0.0,701.0,809.0,713.0,8,10044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173005,2020-07-14T07:21:05.4509560,1,ABR941126007873,1409.0,Siegen-Wittgenstein,Bad Laasphe,05970028,57334,Bad Laasphe,2020-04-16,2009-12-18,35,Hausdach,0,0,5.324,4.300,0.0,688.0,4.30,1450.0,44.0,853.0,802.0,1.0,697.0,810.0,713.0,8,05970
3173006,2020-04-29T09:37:03.4024856,0,ABR903375397990,1409.0,Hochsauerlandkreis,Bestwig,05958008,59909,Bestwig,2020-04-29,2010-07-01,35,Scheune Hof,0,0,16.215,16.215,0.0,688.0,16.50,1450.0,69.0,853.0,802.0,1.0,699.0,808.0,716.0,8,05958
3173007,2020-11-26T08:57:11.0060494,1,ABR985070432688,1403.0,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,92699,Bechtsrieth,2020-04-29,2020-04-28,35,Familie Koller,0,0,29.900,29.900,0.0,689.0,50.00,1450.0,90.0,853.0,803.0,0.0,701.0,809.0,713.0,8,09374
3173008,2020-05-02T10:16:32.7818459,1,ABR925833676461,1405.0,Groß-Gerau,Riedstadt,06433011,64560,Riedstadt,2020-04-29,2016-05-31,35,Hausdach,0,0,3.570,3.000,0.0,689.0,3.00,1449.0,14.0,853.0,803.0,1.0,NaN,809.0,713.0,8,06433


### Data Cleaning - Fehlende Werte

In [66]:
# Welche Spalten weisen noch immer Nan-Values auf? 
nan_count = pv_raw_plz.isna().sum()
print(nan_count )

DatumLetzteAktualisierung                        0
NetzbetreiberpruefungStatus                      0
AnlagenbetreiberMastrNummer                      0
Bundesland                                       0
Landkreis                                        0
Gemeinde                                         0
AGS-8                                            0
Postleitzahl                                     0
Ort                                              0
Registrierungsdatum                              0
Inbetriebnahmedatum                          56661
EinheitBetriebsstatus                            0
NameStromerzeugungseinheit                       9
Weic_nv                                          0
Kraftwerksnummer_nv                              0
Bruttoleistung                                   0
Nettonennleistung                                0
FernsteuerbarkeitNb                          45579
Einspeisungsart                              25903
ZugeordneteWirkleistungWechselr

In [67]:
# Kopie ziehen zur Bearbeitung der fehlenden Daten
pv_raw_m = pv_raw_plz.copy()
pv_raw_m

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,AGS-8,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,len,AGS-5
0,2020-02-20T16:28:35.2508122,1,ABR949444220202,1409.0,Münster,Münster,05515000,48147,Münster,2019-02-01,2007-07-20,35,Photovoltaikanlage ERWin4,0,0,3.960,3.960,0.0,688.0,4.00,1450.0,22.0,853.0,802.0,1.0,699.0,809.0,713.0,8,05515
1,2021-07-15T07:54:37.8637095,1,ABR930055871044,1402.0,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,2013-01-31,35,man-power,0,0,7.410,7.410,0.0,689.0,8.30,1450.0,38.0,853.0,803.0,1.0,699.0,809.0,713.0,8,08136
2,2019-05-28T08:26:38.0316692,1,ABR983885950021,1400.0,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,2016-02-19,35,eigene PVA,0,0,5.040,5.000,0.0,689.0,5.00,1449.0,16.0,853.0,803.0,1.0,699.0,809.0,713.0,8,12063
3,2020-12-07T08:19:07.9860793,1,ABR980052996196,1403.0,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,2016-12-16,35,Photovoltaik EFH Löbert - Mattinger Weg 6,0,0,6.360,6.000,0.0,689.0,6.00,1448.0,24.0,853.0,1535.0,1.0,700.0,809.0,713.0,8,09375
4,2021-01-25T11:18:30.0433603,1,ABR970373690446,1412.0,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,2011-12-08,35,"PV-Anlage 7,2 (Voll)",0,0,7.200,7.200,0.0,689.0,7.60,1450.0,30.0,853.0,802.0,0.0,701.0,809.0,713.0,8,10044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173005,2020-07-14T07:21:05.4509560,1,ABR941126007873,1409.0,Siegen-Wittgenstein,Bad Laasphe,05970028,57334,Bad Laasphe,2020-04-16,2009-12-18,35,Hausdach,0,0,5.324,4.300,0.0,688.0,4.30,1450.0,44.0,853.0,802.0,1.0,697.0,810.0,713.0,8,05970
3173006,2020-04-29T09:37:03.4024856,0,ABR903375397990,1409.0,Hochsauerlandkreis,Bestwig,05958008,59909,Bestwig,2020-04-29,2010-07-01,35,Scheune Hof,0,0,16.215,16.215,0.0,688.0,16.50,1450.0,69.0,853.0,802.0,1.0,699.0,808.0,716.0,8,05958
3173007,2020-11-26T08:57:11.0060494,1,ABR985070432688,1403.0,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,92699,Bechtsrieth,2020-04-29,2020-04-28,35,Familie Koller,0,0,29.900,29.900,0.0,689.0,50.00,1450.0,90.0,853.0,803.0,0.0,701.0,809.0,713.0,8,09374
3173008,2020-05-02T10:16:32.7818459,1,ABR925833676461,1405.0,Groß-Gerau,Riedstadt,06433011,64560,Riedstadt,2020-04-29,2016-05-31,35,Hausdach,0,0,3.570,3.000,0.0,689.0,3.00,1449.0,14.0,853.0,803.0,1.0,NaN,809.0,713.0,8,06433


#### Inbetriebnahmedatum  
Es gibt 56661 fehlende Werte in Inbetriebnahmedatum.

In [68]:
# Registrierungsdatum und Inbetriebnahmedatum von Object in Datetime-Format überführen
pv_raw_m['Inbetriebnahmedatum'] = pd.to_datetime(pv_raw_m['Inbetriebnahmedatum'])
pv_raw_m['Registrierungsdatum'] = pd.to_datetime(pv_raw_m['Registrierungsdatum'])

In [69]:
pv_raw_m.describe()

,NetzbetreiberpruefungStatus,Bundesland,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,Weic_nv,Kraftwerksnummer_nv,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,len
count,3.173010e+06,3.173010e+06,3173010,3116349,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.173010e+06,3.127431e+06,3.147107e+06,3.169709e+06,3.097495e+06,3.088357e+06,3.172953e+06,3.087722e+06,3.108730e+06,3.123924e+06,3.098861e+06,2.764808e+06,3173010.0
mean,7.445441e-01,1.405973e+03,2021-03-27 22:51:23.827659008,2015-11-17 00:23:16.967027200,3.492931e+01,1.249539e-02,5.326173e-05,2.471550e+01,2.187138e+01,1.454788e-01,6.886604e+02,9.951101e+01,1.449476e+03,1.085311e+02,1.024245e+03,8.252088e+02,7.860673e-01,6.993829e+02,8.089033e+02,7.136442e+02,8.0
min,0.000000e+00,1.400000e+03,2019-01-31 00:00:00,1900-01-01 00:00:00,3.100000e+01,0.000000e+00,0.000000e+00,1.000000e-03,1.000000e-03,0.000000e+00,6.880000e+02,1.000000e-03,1.448000e+03,1.000000e+00,8.520000e+02,8.020000e+02,0.000000e+00,6.950000e+02,8.060000e+02,7.130000e+02,8.0
25%,0.000000e+00,1.403000e+03,2020-03-20 00:00:00,2010-06-30 00:00:00,3.500000e+01,0.000000e+00,0.000000e+00,5.200000e+00,4.600000e+00,0.000000e+00,6.880000e+02,5.000000e+00,1.449000e+03,1.800000e+01,8.530000e+02,8.020000e+02,1.000000e+00,6.990000e+02,8.090000e+02,7.130000e+02,8.0
50%,1.000000e+00,1.405000e+03,2020-12-20 00:00:00,2016-04-13 00:00:00,3.500000e+01,0.000000e+00,0.000000e+00,8.640000e+00,8.000000e+00,0.000000e+00,6.890000e+02,8.000000e+00,1.450000e+03,2.900000e+01,8.530000e+02,8.020000e+02,1.000000e+00,6.990000e+02,8.090000e+02,7.130000e+02,8.0
75%,1.000000e+00,1.409000e+03,2022-06-17 00:00:00,2022-03-24 00:00:00,3.500000e+01,0.000000e+00,0.000000e+00,1.260000e+01,1.100000e+01,0.000000e+00,6.890000e+02,1.200000e+01,1.450000e+03,5.000000e+01,8.530000e+02,8.030000e+02,1.000000e+00,7.000000e+02,8.090000e+02,7.130000e+02,8.0
max,1.000000e+00,1.416000e+03,2023-07-01 00:00:00,2023-07-01 00:00:00,3.800000e+01,1.000000e+00,1.000000e+00,2.000000e+05,2.000000e+05,1.000000e+00,6.890000e+02,2.000000e+05,1.450000e+03,1.000000e+06,3.058000e+03,1.535000e+03,1.000000e+00,7.040000e+02,8.110000e+02,7.180000e+02,8.0
std,4.361172e-01,3.834277e+00,NaN,NaN,5.311056e-01,1.110822e-01,7.297870e-03,3.376792e+02,2.916543e+02,3.525830e-01,4.735861e-01,1.860996e+03,8.428187e-01,2.497524e+03,5.668276e+02,1.273903e+02,4.100799e-01,1.496879e+00,7.256231e-01,1.344729e+00,0.0


Die Daten für die Inbetriebnahme liegen zwischen dem 01.01.1900 (?) und dem 01.07.2023   
Die Registrierungsdaten hingegen starten erst am 31.01.2019 und enden am 01.07.2023   

Ab dem 01.07.2017:  Die Anmeldung der Anlagen muss innerhalb eines Monats nach Inbetriebnahme erfolgen.    
Vor dem 01.07.2017: Restrierungsfrist ab 31.01.2021 innerhalb 6 Monate nach Start des Webportals.    
Quelle: https://www.bundesnetzagentur.de/DE/Fachthemen/ElektrizitaetundGas/Monitoringberichte/Marktstammdatenregister/MaStR_node.html   

In [70]:
# Auffüllen der NaN-Werte, indem das Inbetriebnahmedatum mit dem Registrierungsdatum gleichgesetzt wird
pv_raw_m["Inbetriebnahmedatum"] = pv_raw_m["Inbetriebnahmedatum"].fillna(pv_raw_m['Registrierungsdatum'])
# pv_raw_m.isnull().sum()

#### NameStromerzeugungseinheit
Es gibt 9 fehlende Werte in NameStromerzeugungseinheit.

In [71]:
pv_raw_m.groupby('NameStromerzeugungseinheit').count()

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,AGS-8,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,Weic_nv,Kraftwerksnummer_nv,Bruttoleistung,Nettonennleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,len,AGS-5
NameStromerzeugungseinheit,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
!,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2
! 600 W Anlage,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1
! Südseite !,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
!!!,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
!!_keine_Atomkraftwerke_!!,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
🌞 Haus- und Garagendach🌞,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
🌞 SONNENSCHEIN SONNENSCHEIN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
🏠,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


In [72]:
# Es handel sich um individuelle Phantasienamen.
# Ersetzen durch den letzten validen Wert
pv_raw_m['NameStromerzeugungseinheit'] = pv_raw_m['NameStromerzeugungseinheit'].ffill()
# pv_raw_m.isnull().sum()

#### FernsteuerbarkeitNb 
Es gibt 45579 fehlende Werte in FernsteuerbarkeitNb .

In [73]:
# Werte-Verteilung
pv_raw_m['FernsteuerbarkeitNb'].value_counts(normalize=True)

FernsteuerbarkeitNb
0.0    0.854521
1.0    0.145479
Name: proportion, dtype: float64

In [74]:
# Auffüllen der fehlenden Werte anhand der ermittelten Verteilung
pv_raw_m['FernsteuerbarkeitNb'] = pv_raw_m['FernsteuerbarkeitNb'].fillna(pd.Series(np.random.choice([0, 1], 
                                                      p=[0.85, 0.15], size=len(pv_raw_m))))
pv_raw_m.dropna(subset = ['FernsteuerbarkeitNb'], inplace=True)
# pv_raw_m.isnull().sum()
# Quelle: https://stackoverflow.com/questions/38934140/fill-missing-values-by-a-ratio-of-other-values-in-pandas

#### Einspeisungsart
Es gibt 25860 fehlende Werte in Einspeisungsart.

In [75]:
# Werte-Verteilung
pv_raw_m['Einspeisungsart'].value_counts(normalize=True)

Einspeisungsart
689.0    0.660363
688.0    0.339637
Name: proportion, dtype: float64

In [76]:
# Auffüllen der fehlenden Werte anhand der ermittelten Verteilung
pv_raw_m['Einspeisungsart'] = pv_raw_m['Einspeisungsart'].fillna(pd.Series(np.random.choice([689.0 , 688.0], 
                                                      p=[0.66, 0.34], size=len(pv_raw_m))))
pv_raw_m.dropna(subset = ['Einspeisungsart'], inplace=True)
#  pv_raw_m.isnull().sum()
# Quelle: https://stackoverflow.com/questions/38934140/fill-missing-values-by-a-ratio-of-other-values-in-pandas

#### ZugeordneteWirkleistungWechselrichter   
Es gibt 3301 fehlende Werte in ZugeordneteWirkleistungWechselrichter.   
Solaranlagen und Batteriespeicher können auf unterschiedliche Art mit Wechselrichtern verbunden sein.     
Als zugeordnete Wechselrichterleistung ist im MaStR stets die Leistung einzutragen, die typischerweise im Netz    
oder in der Kundenanlage „ankommt“.   
Quelle: https://www.marktstammdatenregister.de/MaStRHilfe/files/regHilfen/210108%20zugeordnete%20Wechselrichterleistung.pdf

In [77]:
# Werte-Verteilung
pv_raw_m['ZugeordneteWirkleistungWechselrichter'].value_counts(normalize=True)

ZugeordneteWirkleistungWechselrichter
10.000     1.088393e-01
0.600      7.052887e-02
8.000      6.358218e-02
5.000      4.106055e-02
6.000      4.052486e-02
               ...     
138.480    3.154864e-07
8.481      3.154864e-07
71.415     3.154864e-07
9.552      3.154864e-07
51.460     3.154864e-07
Name: proportion, Length: 29544, dtype: float64

In [78]:
# Die fehlenden Werte werden mit dem Durchschnittswert ersetzt.
pv_raw_m['ZugeordneteWirkleistungWechselrichter'].fillna(pv_raw_m['ZugeordneteWirkleistungWechselrichter'].mean(), inplace=True)
# pv_raw_m.isnull().sum()

#### GemeinsamerWechselrichterMitSpeicher
Es gibt 75515 fehlende Werte in GemeinsamerWechselrichterMitSpeicher.

In [79]:
# Werte-Verteilung
pv_raw_m['GemeinsamerWechselrichterMitSpeicher'].value_counts(normalize=True)

GemeinsamerWechselrichterMitSpeicher
1450.0    0.70686
1448.0    0.23045
1449.0    0.06269
Name: proportion, dtype: float64

In [80]:
# Auffüllen der fehlenden Werte anhand der ermittelten Verteilung
pv_raw_m['GemeinsamerWechselrichterMitSpeicher'] = pv_raw_m['GemeinsamerWechselrichterMitSpeicher'].fillna(pd.Series(np.random.choice([1450.0 , 1448.0, 1449.0], 
                                                      p=[0.71, 0.23, 0.06], size=len(pv_raw_m))))
pv_raw_m.dropna(subset = ['GemeinsamerWechselrichterMitSpeicher'], inplace=True)
# pv_raw_m.isnull().sum()
# Quelle: https://stackoverflow.com/questions/38934140/fill-missing-values-by-a-ratio-of-other-values-in-pandas

#### AnzahlModule
Es gibt 84607 fehlende Werte in AnzahlModule.

In [81]:
# Extrahieren des Jahres der Inbetriebnahme
pv_raw_m['Inbetriebnahmejahr'] = pv_raw_m['Inbetriebnahmedatum'].dt.year
# Bruttoleistung pro Modul für jedes Jahr der Inbetriebnahme
average_power_per_module = pv_raw_m.groupby('Inbetriebnahmejahr')['Bruttoleistung'].mean() / pv_raw_m.groupby('Inbetriebnahmejahr')['AnzahlModule'].mean()
# Extrapolieren der fehlenden Werte für AnzahlModule
pv_raw_m['AnzahlModule'] = pv_raw_m.apply(lambda row: round(row['Bruttoleistung'] / average_power_per_module[row['Inbetriebnahmejahr']]), axis=1)
#print(pv_raw_m)
#Quelle: ChatGPT (siehe ChatGPT Protokoll Seite 2)

In [82]:
pv_raw_m.isnull().sum()

DatumLetzteAktualisierung                        0
NetzbetreiberpruefungStatus                      0
AnlagenbetreiberMastrNummer                      0
Bundesland                                       0
Landkreis                                        0
Gemeinde                                         0
AGS-8                                            0
Postleitzahl                                     0
Ort                                              0
Registrierungsdatum                              0
Inbetriebnahmedatum                              0
EinheitBetriebsstatus                            0
NameStromerzeugungseinheit                       0
Weic_nv                                          0
Kraftwerksnummer_nv                              0
Bruttoleistung                                   0
Nettonennleistung                                0
FernsteuerbarkeitNb                              0
Einspeisungsart                                  0
ZugeordneteWirkleistungWechselr

#### Lage
Es gibt 57 fehlende Werte in Lage.

In [83]:
# Werte-Verteilung
pv_raw_m['Lage'].value_counts(normalize=True)

Lage
853.0     0.910733
2961.0    0.070905
2484.0    0.013341
852.0     0.005010
3002.0    0.000007
3058.0    0.000004
Name: proportion, dtype: float64

In [84]:
# Auffüllen der fehlenden Werte anhand der ermittelten Verteilung (häufigste 4 Werte)
pv_raw_m['Lage'] = pv_raw_m['Lage'].fillna(pd.Series(np.random.choice([853.0 , 2961.0, 2484.0, 852.0], 
                                                      p=[0.91, 0.07, 0.013, 0.007], size=len(pv_raw_m))))
pv_raw_m.dropna(subset = ['Lage'], inplace=True)
# pv_raw_m.isnull().sum()
# Quelle: https://stackoverflow.com/questions/38934140/fill-missing-values-by-a-ratio-of-other-values-in-pandas

#### Leistungsbegrenzung 
Es gibt 85214 fehlende Werte in Leistungsbegrenzung.

In [85]:
# Werte-Verteilung
pv_raw_m['Leistungsbegrenzung'].value_counts(normalize=True)

Leistungsbegrenzung
802.0     0.647734
803.0     0.311693
1535.0    0.031206
804.0     0.004893
805.0     0.004474
Name: proportion, dtype: float64

In [86]:
# Auffüllen der fehlenden Werte anhand der ermittelten Verteilung (häufigste 3 Werte)
pv_raw_m['Leistungsbegrenzung'] = pv_raw_m['Leistungsbegrenzung'].fillna(pd.Series(np.random.choice([802.0 , 803.0, 1535.0], 
                                                      p=[0.65, 0.31, 0.04], size=len(pv_raw_m))))
pv_raw_m.dropna(subset = ['Leistungsbegrenzung'], inplace=True)
# pv_raw_m.isnull().sum()
# Quelle: https://stackoverflow.com/questions/38934140/fill-missing-values-by-a-ratio-of-other-values-in-pandas

#### EinheitlicheAusrichtungUndNeigungswinkel
Es gibt 64242 fehlende Werte in EinheitlicheAusrichtungUndNeigungswinkel.

In [87]:
# Werte-Verteilung
pv_raw_m['EinheitlicheAusrichtungUndNeigungswinkel'].value_counts(normalize=True)

EinheitlicheAusrichtungUndNeigungswinkel
1.0    0.786067
0.0    0.213933
Name: proportion, dtype: float64

In [88]:
# Auffüllen der fehlenden Werte anhand der ermittelten Verteilung
pv_raw_m['EinheitlicheAusrichtungUndNeigungswinkel'] = pv_raw_m['EinheitlicheAusrichtungUndNeigungswinkel'].fillna(pd.Series(np.random.choice([0, 1], 
                                                      p=[0.21, 0.79], size=len(pv_raw_m))))
pv_raw_m.dropna(subset = ['EinheitlicheAusrichtungUndNeigungswinkel'], inplace=True)
# pv_raw_m.isnull().sum()
# Quelle: https://stackoverflow.com/questions/38934140/fill-missing-values-by-a-ratio-of-other-values-in-pandas

#### Hauptausrichtung
Es gibt 49014 fehlende Werte in Hauptausrichtung.

In [89]:
# Werte-Verteilung
pv_raw_m['Hauptausrichtung'].value_counts(normalize=True)

Hauptausrichtung
699.0    0.501783
700.0    0.191880
698.0    0.131347
701.0    0.060181
704.0    0.058963
697.0    0.039068
696.0    0.005743
702.0    0.005044
695.0    0.004571
703.0    0.001420
Name: proportion, dtype: float64

In [90]:
# Auffüllen der fehlenden Werte anhand der ermittelten Verteilung 
pv_raw_m['Hauptausrichtung'] = pv_raw_m['Hauptausrichtung'].fillna(pd.Series(np.random.choice([699.0, 700.0, 698.0, 701.0, 704.0, 697.0, 696.0, 702.0, 695.0, 703.0], 
                                                      p=[0.50, 0.19, 0.13, 0.06, 0.06, 0.04, 0.007, 0.006, 0.006, 0.001], size=len(pv_raw_m))))
pv_raw_m.dropna(subset = ['Hauptausrichtung'], inplace=True)
# pv_raw_m.isnull().sum()
# Quelle: https://stackoverflow.com/questions/38934140/fill-missing-values-by-a-ratio-of-other-values-in-pandas

#### HauptausrichtungNeigungswinkel
Es gibt 74009 fehlende Werte in HauptausrichtungNeigungswinkel.

In [91]:
# Werte-Verteilung
pv_raw_m['HauptausrichtungNeigungswinkel'].value_counts(normalize=True)

HauptausrichtungNeigungswinkel
809.0    0.578654
808.0    0.228150
810.0    0.171691
806.0    0.010001
807.0    0.008300
811.0    0.003205
Name: proportion, dtype: float64

In [92]:
# Auffüllen der fehlenden Werte anhand der ermittelten Verteilung 
pv_raw_m['HauptausrichtungNeigungswinkel'] = pv_raw_m['HauptausrichtungNeigungswinkel'].fillna(pd.Series(np.random.choice([809.0, 808.0, 810.0, 806.0, 807.0, 811.0], 
                                                      p=[0.58, 0.23, 0.17, 0.01, 0.008, 0.002], size=len(pv_raw_m))))
pv_raw_m.dropna(subset = ['HauptausrichtungNeigungswinkel'], inplace=True)
# pv_raw_m.isnull().sum()
# Quelle: https://stackoverflow.com/questions/38934140/fill-missing-values-by-a-ratio-of-other-values-in-pandas

#### Nutzungsbereich
Es gibt 408138 fehlende Werte in Nutzungsbereich.

In [93]:
# Werte-Verteilung
pv_raw_m['Nutzungsbereich'].value_counts(normalize=True)

Nutzungsbereich
713.0    0.762006
716.0    0.096646
714.0    0.078743
718.0    0.040866
717.0    0.013840
715.0    0.007899
Name: proportion, dtype: float64

In [94]:
# Auffüllen der fehlenden Werte anhand der ermittelten Verteilung 
pv_raw_m['Nutzungsbereich'] = pv_raw_m['Nutzungsbereich'].fillna(pd.Series(np.random.choice([713.0, 716.0, 714.0, 718.0, 717.0, 715.0], 
                                                      p=[0.76, 0.1, 0.08, 0.04, 0.01, 0.01], size=len(pv_raw_m))))
pv_raw_m.dropna(subset = ['Nutzungsbereich'], inplace=True)
pv_raw_m.isnull().sum()
# Quelle: https://stackoverflow.com/questions/38934140/fill-missing-values-by-a-ratio-of-other-values-in-pandas

DatumLetzteAktualisierung                   0
NetzbetreiberpruefungStatus                 0
AnlagenbetreiberMastrNummer                 0
Bundesland                                  0
Landkreis                                   0
Gemeinde                                    0
AGS-8                                       0
Postleitzahl                                0
Ort                                         0
Registrierungsdatum                         0
Inbetriebnahmedatum                         0
EinheitBetriebsstatus                       0
NameStromerzeugungseinheit                  0
Weic_nv                                     0
Kraftwerksnummer_nv                         0
Bruttoleistung                              0
Nettonennleistung                           0
FernsteuerbarkeitNb                         0
Einspeisungsart                             0
ZugeordneteWirkleistungWechselrichter       0
GemeinsamerWechselrichterMitSpeicher        0
AnzahlModule                      

In [95]:
# Verteilung AnlagenbetreiberMastrNummer                 
value_counts = pv_raw_m['AnlagenbetreiberMastrNummer'].value_counts()
print(value_counts)

AnlagenbetreiberMastrNummer
ABR922330502361    781
ABR960080516405    519
ABR967988325672    512
ABR978541747770    295
ABR970056665143    286
                  ... 
ABR994142018880      1
ABR923445362598      1
ABR973671403978      1
ABR910282785642      1
ABR946457251940      1
Name: count, Length: 2606935, dtype: int64


In [96]:
# Anzahl Ausprägungen pro Spalte              
unique_count = pv_raw_m.nunique()
print(unique_count)

DatumLetzteAktualisierung                   3098383
NetzbetreiberpruefungStatus                       2
AnlagenbetreiberMastrNummer                 2606935
Bundesland                                       17
Landkreis                                       382
Gemeinde                                      10563
AGS-8                                         11107
Postleitzahl                                   8160
Ort                                           11480
Registrierungsdatum                            1613
Inbetriebnahmedatum                           11345
EinheitBetriebsstatus                             4
NameStromerzeugungseinheit                  1393075
Weic_nv                                           2
Kraftwerksnummer_nv                               2
Bruttoleistung                                50500
Nettonennleistung                             34614
FernsteuerbarkeitNb                               2
Einspeisungsart                                   2
ZugeordneteW

### Einfügen der Katalogwerte

In [97]:
pv_raw_m.loc[:, 'Bundesland'] = pv_raw_m['Bundesland'].replace(katalog_wert.set_index('Id')['Wert'])
#pv_raw_m.head(5)

In [98]:
pv_raw_m.loc[:, 'Lage'] = pv_raw_m['Lage'].replace(katalog_wert.set_index('Id')['Wert'])
#pv_raw_m.head(5)

In [99]:
pv_raw_m.loc[:, 'EinheitBetriebsstatus'] = pv_raw_m['EinheitBetriebsstatus'].replace(katalog_wert.set_index('Id')['Wert'])
#pv_raw_m.head(5)

In [100]:
pv_raw_m.loc[:, 'Einspeisungsart'] = pv_raw_m['Einspeisungsart'].replace(katalog_wert.set_index('Id')['Wert'])
#pv_raw_m.head(5)

In [101]:
pv_raw_m.loc[:, 'GemeinsamerWechselrichterMitSpeicher'] = pv_raw_m['GemeinsamerWechselrichterMitSpeicher'].replace(katalog_wert.set_index('Id')['Wert'])
#pv_raw_m.head(5)

In [102]:
pv_raw_m.loc[:, 'Leistungsbegrenzung'] = pv_raw_m['Leistungsbegrenzung'].replace(katalog_wert.set_index('Id')['Wert'])
#pv_raw_m.head(5)

In [103]:
pv_raw_m.loc[:, 'Hauptausrichtung'] = pv_raw_m['Hauptausrichtung'].replace(katalog_wert.set_index('Id')['Wert'])
#pv_raw_m.head(5)

In [104]:
pv_raw_m.loc[:, 'HauptausrichtungNeigungswinkel'] = pv_raw_m['HauptausrichtungNeigungswinkel'].replace(katalog_wert.set_index('Id')['Wert'])
#pv_raw_m.head(5)

In [105]:
pv_raw_m.loc[:, 'Nutzungsbereich'] = pv_raw_m['Nutzungsbereich'].replace(katalog_wert.set_index('Id')['Wert'])
#pv_raw_m.head(5)

In [106]:
# nicht benötigte Spalten löschen
drop_cols = ['len', 'Nettonennleistung', 'NameStromerzeugungseinheit', 'Weic_nv', 'Kraftwerksnummer_nv']
pv_raw_m = pv_raw_m.drop(drop_cols, axis=1)
pv_raw_m

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,AGS-8,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,Bruttoleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,AGS-5,Inbetriebnahmejahr
0,2020-02-20T16:28:35.2508122,1,ABR949444220202,Nordrhein-Westfalen,Münster,Münster,05515000,48147,Münster,2019-02-01,2007-07-20,In Betrieb,3.960,0.0,Volleinspeisung,4.00,Kein Stromspeicher vorhanden,32,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Süd,20 - 40 Grad,Haushalt,05515,2007
1,2021-07-15T07:54:37.8637095,1,ABR930055871044,Baden-Württemberg,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,2013-01-31,In Betrieb,7.410,0.0,Teileinspeisung (einschließlich Eigenverbrauch),8.30,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,08136,2013
2,2019-05-28T08:26:38.0316692,1,ABR983885950021,Brandenburg,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,2016-02-19,In Betrieb,5.040,0.0,Teileinspeisung (einschließlich Eigenverbrauch),5.00,Stromspeicher vorhanden - kein gemeinsamer Wec...,22,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,12063,2016
3,2020-12-07T08:19:07.9860793,1,ABR980052996196,Bayern,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,2016-12-16,In Betrieb,6.360,0.0,Teileinspeisung (einschließlich Eigenverbrauch),6.00,Stromspeicher vorhanden - gemeinsamer Wechselr...,27,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, sonstige",1.0,Süd-West,20 - 40 Grad,Haushalt,09375,2016
4,2021-01-25T11:18:30.0433603,1,ABR970373690446,Saarland,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,2011-12-08,In Betrieb,7.200,0.0,Teileinspeisung (einschließlich Eigenverbrauch),7.60,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,0.0,West,20 - 40 Grad,Haushalt,10044,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173005,2020-07-14T07:21:05.4509560,1,ABR941126007873,Nordrhein-Westfalen,Siegen-Wittgenstein,Bad Laasphe,05970028,57334,Bad Laasphe,2020-04-16,2009-12-18,In Betrieb,5.324,0.0,Volleinspeisung,4.30,Kein Stromspeicher vorhanden,40,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Ost,< 20 Grad,Haushalt,05970,2009
3173006,2020-04-29T09:37:03.4024856,0,ABR903375397990,Nordrhein-Westfalen,Hochsauerlandkreis,Bestwig,05958008,59909,Bestwig,2020-04-29,2010-07-01,In Betrieb,16.215,0.0,Volleinspeisung,16.50,Kein Stromspeicher vorhanden,98,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Süd,40 - 60 Grad,Landwirtschaft,05958,2010
3173007,2020-11-26T08:57:11.0060494,1,ABR985070432688,Bayern,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,92699,Bechtsrieth,2020-04-29,2020-04-28,In Betrieb,29.900,0.0,Teileinspeisung (einschließlich Eigenverbrauch),50.00,Kein Stromspeicher vorhanden,90,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",0.0,West,20 - 40 Grad,Haushalt,09374,2020
3173008,2020-05-02T10:16:32.7818459,1,ABR925833676461,Hessen,Groß-Gerau,Riedstadt,06433011,64560,Riedstadt,2020-04-29,2016-05-31,In Betrieb,3.570,0.0,Teileinspeisung (einschließlich Eigenverbrauch),3.00,Stromspeicher vorhanden - kein gemeinsamer Wec...,15,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,06433,2016


In [107]:
#pv_raw_m.isna().sum()

In [108]:
# Bereinigten Datensatz als Pickle speichern
pv_raw_clean = pv_raw_m.copy()
pv_raw_clean.to_pickle('marktstammdaten.pkl')
# Quelle: https://www.statology.org/pandas-save-dataframe/

# Export als SQLite Datenbank

In [109]:
# SQLite Datenbank Pfad 
SQLITE_FILE_PATH = os.path.join("..", "output", "dwh.sqlite3")

In [110]:
# Connection zur Datenbank 
con = sqlite3.connect(SQLITE_FILE_PATH)

## Tabelle exportieren

In [111]:
# Marktstammdaten-Tabelle anlegen
pv_raw_clean.to_sql("marktstammdaten", con=con, if_exists="replace", index=False)

3173010

In [112]:
# Verbindung zur Datenbank trennen
con.close()